In [1]:
import pandas as pd

melb_df = pd.read_csv('data/melb_data_ps.csv', sep = ',')

In [2]:
melb_df['Address'].nunique()

13378

In [3]:
## Извлечём из признака адреса характеристику подтипа улицы (улица, шоссе, авеню, бульвар)
# На вход данной функции поступает строка с адресом.
def get_street_type(address):
    # Создаём список географических пометок exclude_list.
    exclude_list = ['N', 'S', 'W', 'E']
    # Метод split() разбивает строку на слова по пробелу.
    # В результате получаем список слов в строке и заносим его в переменную address_list.
    address_list = address.split(' ')
    # Обрезаем список, оставляя в нём только последний элемент,
    # потенциальный подтип улицы, и заносим в переменную street_type. 
    street_type = address_list[-1]
    # Делаем проверку на то, что полученный подтип является географической пометкой.
    # Для этого проверяем его на наличие в списке exclude_list. 
    if street_type in exclude_list:
    # Если переменная street_type является географической пометкой,
    # переопределяем её на второй элемент с конца списка address_list.
        return address_list[-2]
    return street_type

In [6]:
street_types = melb_df['Address'].apply(get_street_type)
street_types

0        St
1        St
2        St
3        La
4        St
         ..
13575    Cr
13576    Dr
13577    St
13578    St
13579    St
Name: Address, Length: 13580, dtype: object